### **LLM Bind Tools**

- LLM에 Tools 적용

**참고**
- 도구를 정의할 때 `@tool` 데코레이터를 사용하여 도구를 정의
- docstring 은 가급적 영어로 작성하는 것을 권장

In [3]:
import re
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.agents import tool

a = [1, 2, 3, 4, 5]
a

In [3]:
# 단어의 길이 반환
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# 두 숫자를 더함
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

# 네이버 뉴스 크롤링
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]

### **bind_tools() 로 LLM 에 도구 바인딩**

- name : 도구 이름
- args : 도구에 전달되는 인자

In [4]:
from langchain_openai import ChatOpenAI

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [5]:
llm_with_tools.invoke("what is 3 + 2?").tool_calls

[{'name': 'add_function',
  'args': {'a': 3, 'b': 2},
  'id': 'call_c9GzzYUw2vOQfutSWkfmTP4w',
  'type': 'tool_call'}]

In [6]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'teddynote'?")
tool_call_results

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]

In [7]:
print(tool_call_results, end="\n\n==========\n\n")

# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]

single_result

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]




{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}

In [8]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회
    for tool_call_result in tool_call_results:
        
        # 도구의 이름과 인자를 추출
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        # 도구 이름과 일치하는 도구를 찾아 실행
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)
        print(matching_tool, end="\n\n")
        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


# 도구 호출 실행
# 이전에 얻은 tool_call_results를 인자로 전달하여 함수를 실행
execute_tool_calls(tool_call_results)

name='get_word_length' description='Returns the length of a word.' args_schema=<class 'langchain_core.utils.pydantic.get_word_length'> func=<function get_word_length at 0x0000022461FA9800>

[실행도구] get_word_length [Argument] {'word': 'teddynote'}
[실행결과] 9


In [9]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

chain.invoke("What is the length of the word 'teddynote'?")

name='get_word_length' description='Returns the length of a word.' args_schema=<class 'langchain_core.utils.pydantic.get_word_length'> func=<function get_word_length at 0x0000022461FA9800>

[실행도구] get_word_length [Argument] {'word': 'teddynote'}
[실행결과] 9
